In [1]:
import glob
import os
import matplotlib as mlib
import matplotlib.pyplot as plt
import numpy as np
from numpy import genfromtxt
from numpy import loadtxt
import re
import csv
import matplotlib.colors as mcolors
import matplotlib.cm as cm
import matplotlib.patches as mpatches

In [2]:
#constants


In [3]:
# d0 for lectrons and muons separately\n
def impact_d0(folder):
    #reading pythia output
    files = sorted(glob.glob( folder+'/*'))
    print(files)

    
    open("hepmc_output/d0_e.dat","w")
    open("hepmc_output/d0_mu.dat","w")
    open("hepmc_output/pT_e.dat","w")
    open("hepmc_output/pT_mu.dat","w")
    #open hepmc file
    for filenum, file in enumerate(files):
        hepmc=[]
        d0_l=[]
        d0_e=[]
        d0_mu=[]
        pT_e=[]
        pT_mu=[]
        print(file)
        with open (file, 'r') as f:
            for rownum,row in enumerate(csv.reader(f,delimiter=' ')):
                if (len(row)>4):
                    hepmc.append(row)

        print(len(hepmc)) 
        
        counte=0
        countmu=0
        countevents=0
        countV0=0
        for rownum ,row in enumerate(hepmc):
            #print(hepmc[rownum][0])
            #finding vertex
            if (row[0]=='E'):
                countevents+=1
            if (row[0]=='V'):
                #print('V=',row)
                ip=rownum+1
                #finding electron/muon related to this vertex (knowing that max 3 particles can originate from it)
                for i in range(0,2): 
                    #print(hepmc[ip+i])
                    if (ip+i<len(hepmc)):
                        if (hepmc[ip+i][2]=='11'):
                            counte+=1
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)

                            #calculating |d0|
                            VT=np.sqrt(V[0]**2+V[1]**2)
                            PT=np.sqrt(P[0]**2+P[1]**2)
                            
                            if ((V[0]!=0)&(V[1]!=0)&(V[2]!=0)): 
                                cTheta=(V[0]*P[0]+V[1]*P[1])/(VT*PT)
                                
                                d0_e.append(PT*np.sqrt(1-cTheta**2))
                                
                            else: 
                                countV0+=1
                                d0_e.append(0)
                                
                            pT_e.append(PT)    
                            
                            
                        if (hepmc[ip+i][2]=='13'):
                            countmu+=1
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)
                            #print(P)

                            #calculating |d0|
                            VT=np.sqrt(V[0]**2+V[1]**2)
                            PT=np.sqrt(P[0]**2+P[1]**2)
                            
                            if ((V[0]!=0)&(V[1]!=0)&(V[2]!=0)):  
                                cTheta=(V[0]*P[0]+V[1]*P[1])/(VT*PT)
                                
                                d0_mu.append(PT*np.sqrt(1-cTheta**2))
                                
                            else: 
                                countV0+=1
                                d0_mu.append(0)
                            
                            pT_mu.append(PT)

                        i+=1
        #print(d0_l)
    
        print("# events = {}".format(countevents)+"# e = {}".format(counte)+"# mu = {}".format(countmu)\
              +"# V(primary) = {}".format(countV0))
        
        #Saving d0 to file
        with open("hepmc_output/d0_e.dat", 'a+') as foute:
                        #fout.write('# d0 [mm]' + '\n')
                        np.savetxt(foute, d0_e, fmt='%7f')   
                    
        with open("hepmc_output/d0_mu.dat", 'a+') as foutmu:
                        #fout.write('# d0 [mm]' + '\n')
                        np.savetxt(foutmu, d0_mu, fmt='%7f')
                    
        #Saving pT to file
        with open("hepmc_output/pT_e.dat", 'a+') as foute:
                        np.savetxt(foute, pT_e, fmt='%7f')   
                    
        with open("hepmc_output/pT_mu.dat", 'a+') as foutmu:
                        np.savetxt(foutmu, pT_mu, fmt='%7f')

In [4]:
# lifetime of chi_+
def lifetime(folder,outputfolder):
    #reading pythia output
    files = sorted(glob.glob( folder+'/*'))
    print(files)

    os.makedirs(os.path.dirname(outputfolder+"/ctau_plus.dat"), exist_ok=True)
    open(outputfolder+"/ctau_plus.dat","w")
    open(outputfolder+"/DM.dat","w")
    open(outputfolder+"/beta_plus.dat","w")
    open(outputfolder+"/gamma_plus.dat","w")
    open(outputfolder+"/betagamma_plus.dat","w")
    open(outputfolder+"/pT_plus.dat","w")
    
    #open hepmc file
    for filenum, file in enumerate(files):
        hepmc=[]
        ctau_plus=[]
        beta_plus=[]
        gamma_plus=[]
        betagamma_plus=[]
        pT_plus=[]
        DM=[]

        print(file)
        with open (file, 'r') as f:
            for rownum,row in enumerate(csv.reader(f,delimiter=' ')):
                if (len(row)>4):
                    hepmc.append(row)

        print(len(hepmc)) 

        countevents=0
        countV0=0
        for rownum ,row in enumerate(hepmc):
            #print(hepmc[rownum][0])
            #finding vertex
            if (row[0]=='E'):
                countevents+=1
            
            #calculating beta and gamma factors of chi_plus
            if (row[2]=='9000007') or (row[2]=='-9000007'):
                
                plus=np.array(row[3:7], float)
                #print(plus)
                
                beta=np.sqrt(plus[0]**2+plus[1]**2+plus[2]**2)/plus[3]
                beta_plus.append(beta)
                gamma_plus.append(1/np.sqrt(1-beta**2))
                betagamma_plus.append(beta*1/np.sqrt(1-beta**2))
                pT_plus.append(np.sqrt(plus[0]**2+plus[1]**2))
                
            
            if (row[0]=='V'):
                #print('V=',row)
                ip=rownum+1
                #finding DM related to this vertex (knowing that max 3 particles can originate from it)
                for i in range(0,2): 
                    #print(hepmc[ip+i])
                    if (ip+i<len(hepmc)):
                        if (hepmc[ip+i][2]=='9000005'):
                            DM.append(hepmc[ip+i])
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)

                            #calculating lifetime and time of flight in lab frame
                            ctau_plus.append(np.sqrt(V[0]**2+V[1]**2+V[2]**2))
            
                              

                        i+=1
        #print(d0_l)
    
        #print("# events = {}".format(countevents)+"# e = {}".format(counte)+"# mu = {}".format(countmu)\
        #      +"# V(primary) = {}".format(countV0))
        
        #Saving to files 
        with open(outputfolder+"/ctau_plus.dat", 'a+') as fout:
                        np.savetxt(fout, ctau_plus, fmt='%7f') 
                    
        with open(outputfolder+"/beta_plus.dat", 'a+') as fout:
                        np.savetxt(fout, beta_plus, fmt='%7f') 
                    
        with open(outputfolder+"/gamma_plus.dat", 'a+') as fout:
                        np.savetxt(fout, gamma_plus, fmt='%7f') 
                
        with open(outputfolder+"/betagamma_plus.dat", 'a+') as fout:
                        np.savetxt(fout, betagamma_plus, fmt='%7f') 
                
        with open(outputfolder+"/pT_plus.dat", 'a+') as fout:
                        np.savetxt(fout, pT_plus, fmt='%7f') 
                
        #print(DM)            
        #with open(outputfolder+"/DM.dat", 'a+') as fout:
        #                fout.write(DM)

In [5]:
# lifetime of chi_+
def lifetimeAngle(folder,outputfolder,tanmin,tanmax):
    #reading pythia output
    files = sorted(glob.glob( folder+'/*'))
    print(files)

    os.makedirs(os.path.dirname(outputfolder+"/ctau_plus.dat"), exist_ok=True)
    open(outputfolder+"/ctau_plus.dat","w")
    #open hepmc file
    for filenum, file in enumerate(files):
        hepmc=[]
        ctau_plus=[]

        print(file)
        with open (file, 'r') as f:
            for rownum,row in enumerate(csv.reader(f,delimiter=' ')):
                if (len(row)>4):
                    hepmc.append(row)

        print(len(hepmc)) 
        
        counte=0
        countmu=0
        countevents=0
        countAngle=0
        for rownum ,row in enumerate(hepmc):
            #print(hepmc[rownum][0])
            #finding vertex
            if (row[0]=='E'):
                countevents+=1
            if (row[0]=='V'):
                #print('V=',row)
                ip=rownum+1
                #finding DM related to this vertex (knowing that max 3 particles can originate from it)
                for i in range(0,2): 
                    #print(hepmc[ip+i])
                    if (ip+i<len(hepmc)):
                        if (hepmc[ip+i][2]=='9000005'):
                            countDM+=1
                            #print(hepmc[ip+i][2:6])
                            V=np.array(row[3:6], float)
                            P=np.array(hepmc[ip+i][3:6], float)

                            #calculating ctau in the range of angles:
                            VT=np.sqrt(V[0]**2+V[1]**2)
                            tantheta=VT/np.abs(V[2])
                            
                            if ((tanmin<tantheta)&(tantheta<tanmax)):
                                countAngle+=1
                                ctau_plus.append(np.sqrt(V[0]**2+V[1]**2+V[2]**2))

                              

                        i+=1
        #print(d0_l)
    
        print("# events = {}".format(countevents)+"# in range = {}".format(countAngle))
        
        #Saving ctau to file 
        
        with open(outputfolder+"/ctau_plus.dat", 'a+') as fout:
                        #fout.write('# d0 [mm]' + '\n')
                        np.savetxt(fout, ctau_plus, fmt='%7f')   

In [6]:
#choosing var1 & var2 with respect to cut on var1

def chooseWithCut(file1,file2,variable1,variable2,output_folder,cut_file1):
    var1 = np.array(genfromtxt(file1))
    var2 = np.array(genfromtxt(file2))
    
    print(var2[var1>cut_file1])
    
    var1New=var1[var1>cut_file1]
    var2New=var2[var1>cut_file1]
    
    oute=output_folder+"_cut/"
    
    os.makedirs(os.path.dirname(oute), exist_ok=True)
    
    with open(out+variable1+'.dat', 'w') as fout1:
        np.savetxt(fout1, var1New, fmt='%7f')
    
    with open(out+variable1+'.dat', 'w') as fout2:
        np.savetxt(fout2, var2New, fmt='%7f')

In [24]:
chooseWithCut('histos/pT_pythia/pT_e.dat','histos/d0_real/d0_e.dat','pT','d0','e_cut',10)

[0. 0. 0. ... 0. 0. 0.]


In [25]:
chooseWithCut('histos/pT_pythia/pT_mu.dat','histos/d0_real/d0_mu.dat','pT','d0','mu_cut',10)

[0. 0. 0. ... 0. 0. 0.]


In [28]:
chooseWithCut('mu_cut_cutd0','mu_cut_cutpT','d0','pT','mu_cutd0',0.1)

[ 22.48824   10.95337   11.590138  26.533526  25.661871  33.347054
  47.180525  16.65553   34.454754  38.943485  51.863642  21.265549
  41.956833  10.046094  14.016573  10.170316  21.103116  16.063442
  15.81082   16.122646  15.000018 155.647521  11.611274  30.606574
  62.31787   17.306301  14.10992   15.199604  12.850379  23.766619
  12.360929  43.298761 175.8681    17.53558   49.342838  35.68634
  15.541663  23.859052  20.732581  15.205266  19.28075   13.890731
  20.855286  12.019492  13.51624   10.376472  12.002468  25.395985
  29.198502  16.549653  10.107551  20.459342  14.301696  10.003159
  18.519944  14.692021  25.919166  12.330411  49.719596  12.664613
  15.921522  10.202953  10.098852  20.235758  13.762713  10.322614
  26.013009  13.441925  48.884146  17.22727   15.377062 161.693488
  13.431368  11.921576  19.780699  17.602177  11.816107  14.882669
  10.774668  10.456395  11.115739  14.626191  62.65736   41.278775
  10.765434  10.472835 116.073597  11.347817  25.354522  19.462

In [29]:
chooseWithCut('e_cut_cutd0','e_cut_cutpT','d0','pT','e_cutd0',0.1)

[16.510124 74.856627 18.278645 14.420051 26.081339 10.052906 38.009223
 16.739465 11.62326  45.46512  30.778797 45.345274 10.750851 11.731046
 25.356011 22.856074 12.454772 62.04214  12.199269 29.058587 15.646389
 53.116439 17.570276 30.93712  23.731003 11.63663  29.715172 14.440059
 16.775751 25.997271 19.310832 74.075755 18.395727 15.509746 17.6158
 17.415424 21.572153 10.222176 17.33624  19.72876  22.507979 39.546741
 13.779528 11.504213 62.639314 12.236002 35.496261 15.499421 10.948644
 28.568007 15.029108 28.841759 10.004351 32.889439 19.831541 39.478919
 18.178901 25.938783 19.192789 27.408195 22.941176 49.157454 13.094178
 19.280946 18.744305 42.862113 18.895644 12.078462 13.824418 19.172304
 19.196477 10.800636 50.351624 10.672964 46.990916 10.935708 31.722816
 67.992224 42.78604  62.086922 30.144308 11.193355 73.67114  11.976523
 20.887525 11.266953 53.870963 10.685612 13.26258  12.112184 12.836717
 28.599665 10.139282 15.554587 12.227823 28.774732 36.914607 12.981845
 10.9102

In [38]:
impact_d0('hepmc_large_split')

[]


In [7]:
impact_d0('hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02')

['hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaaa', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaab', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaac', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaad', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaae', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaaf', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaag', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaah', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaai', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaaj', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaak', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaal', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaam', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaan', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaao', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaap', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaaq', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaar', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaas']
hepmc_rr1.00e-08_mS3.04e+02_mT

KeyboardInterrupt: 

In [8]:
lifetime('hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02',"hepmc_output_rr1e-8")

['hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaaa', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaab', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaac', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaad', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaae', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaaf', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaag', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaah', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaai', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaaj', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaak', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaal', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaam', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaan', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaao', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaap', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaaq', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaar', 'hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02/pythiaas']
hepmc_rr1.00e-08_mS3.04e+02_mT

In [9]:
lifetime("hepmc_rr1.00e-09_mS3.04e+02_mT3.24e+02","hepmc_output_rr1e-9")

['hepmc_rr1.00e-09_mS3.04e+02_mT3.24e+02/tag_1_pythia8_events.hepmc']
hepmc_rr1.00e-09_mS3.04e+02_mT3.24e+02/tag_1_pythia8_events.hepmc
5590413


In [ ]:
lifetimeAngle("hepmc_rr1.00e-08_mS3.04e+02_mT3.24e+02",'hepmc_output_rr1e-8_Angle-0.2-0.3',0.22,0.28)

In [19]:
chooseWithCut('hepmc_output/pT_e.dat','hepmc_output/d0_e.dat','pT','d0','histos/d0_ctau2cm_pT10_e',10)

[ 7.974428 11.289619  3.867146 ... 12.838999 12.781385  0.875444]


In [20]:
chooseWithCut('hepmc_output/pT_mu.dat','hepmc_output/d0_mu.dat','pT','d0','histos/d0_ctau2cm_pT10_mu',10)

[4.770571 0.31015  0.310149 ... 0.492623 0.263741 6.539914]


In [21]:
chooseWithCut('hepmc_output/d0_e.dat','hepmc_output/pT_e.dat','d0','pT','histos/cut_d0',100)

[]


In [27]:
# devinding lifetime in 3 peaces

var1 = np.array(genfromtxt('hepmc_output/ctau_plus.dat'))


low=var1[var1<0.1]
middle=var1[(0.1<var1)&(var1<10)]
high=var1[10<var1]


out="histos/lifetime_split"

os.makedirs(os.path.dirname(out), exist_ok=True)

with open(out+'/ctau_low.dat', 'w') as fout:
    np.savetxt(fout, low, fmt='%7f')
    
with open(out+'/ctau_middle.dat', 'w') as fout:
    np.savetxt(fout, middle, fmt='%7f')
    
with open(out+'/ctau_high.dat', 'w') as fout:
    np.savetxt(fout, high, fmt='%7f')